In [1]:
pip install simpy networkx

In [10]:
import simpy
import networkx as nx
import random

# Define a frame class to represent the frames being transferred in the network
class Frame:
    storage_time_limit = 10  # Maximum allowed storage time

    def __init__(self, env, source, destination, frame_id):
        self.env = env  # Simulation environment
        self.source = source  # Source node of the frame
        self.destination = destination  # Destination node of the frame
        self.creation_time = env.now  # Timestamp of frame creation
        self.status = 'in transit' # Status of the frame: 'in transit', 'discarded' or 'arrived'
        self.path_traveled = [source]  # Path the frame has traveled
        self.frame_id = frame_id  # Unique identifier for the frame
        self.time_in_storage = 0  # Time spent in storage
        self.queue_entry_time = None  # Time when the frame is added to the queue
        self.guard_time = 3  # Guard time for frame integrity
        self.payload_time = 1
        self.temporal_frame_length = self.guard_time + self.payload_time  # Length of the frame in time units
        self.num_times_entered_storage = 0 # number of times a frame has entered storage
        self.time_in_queue = 0

    def update_time_in_queue(self, time_in_queue):
        self.time_in_queue += time_in_queue

    def update_frame_from_processing(self, pretransmission_delay, frame_length_before_processing):
        '''
        pretransmission delay: queuing and header processing delay
        transmission delay: temporal frame length
        '''
        # checking for invalid times
        if pretransmission_delay < 0 or self.guard_time < 0:
            raise ValueError("Invalid time value")

        # if the guard time is larger then it is reduced, no time in storage
        elif self.guard_time >= pretransmission_delay:
            self.guard_time -= pretransmission_delay
            print(f'FRAME {self.frame_id}, GUARD TIME {self.guard_time}')
            self.temporal_frame_length -= pretransmission_delay

        # if the guard time is 0 then the frame is in storage the whole time
        # tramsission delay is halfed bc its the avg time part of the payload will spend in storage
        elif round(self.guard_time,1) == 0:
            self.time_in_storage += (pretransmission_delay + frame_length_before_processing / 2)
            print(f'FRAME {self.frame_id} TRANSMISSION DELAY:', frame_length_before_processing)
            print(f'FRAME {self.frame_id} PRETRANSMISSION DELAY:', pretransmission_delay)
            self.num_times_entered_storage += 1

        # if the time delay is less then the guard time then the guard time
        # becomes zero and the packet is in storage after the guard time is used up
        elif self.guard_time < pretransmission_delay:
            self.temporal_frame_length -= self.guard_time
            self.time_in_storage += (pretransmission_delay - self.guard_time + self.temporal_frame_length / 2)
            #print(f'FRAME {self.frame_id} TRANSMISSION DELAY:', transmission_delay)
            #print(f'FRAME {self.frame_id} PRETRANSMISSION DELAY:', pretransmission_delay)
            self.num_times_entered_storage += 1
            self.guard_time = 0
            print(f'FRAME {self.frame_id}, GUARD TIME {self.guard_time}')


        # checking if storage time limit has been reached
        if self.time_in_storage > Frame.storage_time_limit:
            self.status = 'discarded'

class NetworkNode:
    header_processing = 0

    def __init__(self, env, name, node_type, k=2):
        self.env = env  # Simulation environment
        self.name = name  # Name of the node
        self.node_type = node_type  # Type of node: 'sender', 'router', or 'receiver'
        self.queue = simpy.Store(env)  # Queue to store frames
        self.resource = simpy.Resource(env, capacity=k)  # Resource to handle k simultaneous processes

    def process_frames(self, print_status):
        while True:
            frame = yield self.queue.get()  # Get the next frame from the queue
            with self.resource.request() as request:
                yield request  # Wait until the resource is available

                queue_exit_time = self.env.now  # Record the time when the frame is retrieved from the queue
                time_in_queue = queue_exit_time - frame.queue_entry_time  # Calculate time spent in the queue
                frame.update_time_in_queue(time_in_queue)
                if print_status:
                    print(f"Time {self.env.now}: Frame {frame.frame_id} spent {time_in_queue} time units in queue at {self.name}")

                if frame.status == 'discarded':
                    if print_status:
                        print(f"Time {self.env.now}: Frame {frame.frame_id} discarded at {self.name}")
                    continue  # Skip further processing if frame is discarded

                if self.node_type in ['router', 'receiver']:
                    # Simulate header processing delay for routers and receivers
                    yield self.env.timeout(NetworkNode.header_processing)

                    # update the frame with the total delay time at this node
                    total_preprocessing_delay = NetworkNode.header_processing + time_in_queue

                    frame_length_before_processing = frame.temporal_frame_length
                    frame.update_frame_from_processing(total_preprocessing_delay, frame_length_before_processing)

                    frame_length_after_processing = frame.temporal_frame_length
                    yield self.env.timeout(frame_length_after_processing)

                    if frame.status == 'discarded':
                        if print_status:
                            print(f"Time {self.env.now}: Frame {frame.frame_id} discarded at {self.name}")
                        continue  # Skip further processing if frame is discarded

                if self.name != frame.destination:
                    # If the current node is not the destination, forward the frame
                    self.env.process(route_frame(self.env, frame, network, self, print_status))
                else:
                    # If the current node is the destination, deliver the frame
                    frame.status = 'arrived'
                    if print_status:
                        print(f"Time {self.env.now}: Frame {frame.frame_id} delivered at {self.name}, from {frame.source} to {frame.destination}")

    def put_frame_in_queue(self, frame):
        frame.queue_entry_time = self.env.now  # Record the time when the frame is added to the queue
        self.queue.put(frame)

# Define a function to generate frames at a given source node
def generate_frames(env, source_node, destination, interval, max_packets, print_status):
    packet_count = 0  # Counter for the number of packets generated
    while packet_count < max_packets:
        yield env.timeout(interval)  # Wait for the next frame generation interval
        # Create a new frame with example attributes
        frame = Frame(env, source_node.name, destination, packet_count + 1)
        source_node.put_frame_in_queue(frame)  # Put the frame in the source node's queue using the correct method
        if print_status:
            print(f"\nTime {env.now}: Frame {frame.frame_id} created at {source_node.name}, destined for {destination}")
        frames.append(frame)  # Collect the frame for visualization
        packet_count += 1  # Increment the packet counter

def route_frame(env, frame, network, current_node, print_status):
    # Calculate the shortest path from the current node to the destination
    all_paths = list(nx.all_shortest_paths(network, source=current_node.name, target=frame.destination, weight='weight'))
    path = random.choice(all_paths)

    # Determine the next hop in the path
    next_hop = path[1]
    # Get the next node object from the dictionary
    next_node = node_dict[next_hop]

    yield env.timeout(0)

    frame.path_traveled.append(next_hop)  # Record the path traveled
    if print_status:
        print(f"\nTime {env.now}: Frame {frame.frame_id} forwarded from {current_node.name} to {next_hop}")
    # Put the frame in the next node's queue
    next_node.put_frame_in_queue(frame)

def create_linear_graph(network, num_nodes):
    nodes = []
    for i in range(num_nodes):
        node = f'Node_{i}'
        nodes.append(node)
        network.add_node(node)
        # Connect each node to the next one
        if i > 0:
            network.add_edge(nodes[i - 1], node, weight=1)
            network.add_edge(node, nodes[i - 1], weight=1)
    return nodes

def initialize_simulation(num_nodes, sender, receiver):
    """
    num_nodes=int
    sender = int, index of sender node
    receiver = int, index of receiver node
    """
    global env, network, node_dict, frames
    env = simpy.Environment()
    network = nx.Graph()
    frames = []

    # Create a linear graph of nodes
    nodes = create_linear_graph(network, num_nodes)

    # Define node types (for simplicity, all nodes are routers except the sender and receiver)
    node_types = {}
    for i in range(num_nodes):
        if i == sender:
            node_types[f'Node_{i}'] = 'sender'
        elif i == receiver:
            node_types[f'Node_{i}'] = 'receiver'
        else:
            node_types[f'Node_{i}'] = 'router'

    # Create network node objects and store them in a dictionary
    node_dict = {name: NetworkNode(env, name, node_types[name], k=1) for name in nodes}


def retry_discarded_frames(sender, receiver, probability, max_retries, print_status):
    retries = 0
    while retries < max_retries:
        discarded_frames = [frame for frame in frames if frame.status == 'discarded' or frame.path_traveled[-1] != f'Node_{receiver}']

        if not discarded_frames:
            break  # Exit the loop if no frames need to be resent

        print('\n')
        for frame in discarded_frames:
            # Reset parameters
            frame.creation_time = frame.env.now
            frame.status = 'in transit' if random.random() < probability else 'discarded'
            frame.path_traveled = [frame.source]  # Reset path traveled
            frame.time_in_storage = 0
            frame.guard_time = 3
            frame.temporal_frame_length = frame.guard_time + frame.payload_time

            if print_status:
                print(f"Retrying frame {frame.frame_id}, new status: {frame.status}")
            node_dict[f'Node_{sender}'].put_frame_in_queue(frame)  # Resend frame from the sender node

        print('\n')
        # Run the simulation again for the resent frames
        while any(frame.status == 'in transit' for frame in frames):
            env.step()

        retries += 1

    return retries



def run_simulation(max_packets, interval, num_nodes, sender, receiver, max_retries=10, print_status=True):
    initialize_simulation(num_nodes, sender, receiver)

    # Start the frame generation process at the sender node, creating frames destined for the receiver node
    env.process(generate_frames(env, node_dict[f'Node_{sender}'], f'Node_{receiver}', interval=interval, max_packets=max_packets, print_status=print_status))

    # Start the frame processing process for each node in the network
    for node in node_dict.values():
        env.process(node.process_frames(print_status))

    # Run the simulation until all packets are generated and processed
    while len(frames) < max_packets or any(frame.status == 'in transit' for frame in frames):
        env.step()

    # Retry sending any discarded packets max_retries times
    retries = retry_discarded_frames(sender, receiver, probability, max_retries, print_status)

    # Count packets received and not received
    received_count = sum(1 for frame in frames if frame.status == 'arrived')
    not_received_count = len(frames) - received_count
    discarded_frame_ids = [frame.frame_id for frame in frames if frame.status == 'discarded']
    simulation_time = env.now - interval


    if not_received_count != 0:
        raise UserWarning(f"All packets not sent after {max_retries} retries")

    """
    print(f"\nSimulation completed. Packets received: {received_count}, Packets not received: {not_received_count}")
    print(f"Retries: {retries}")
    print(f"Simulation time: {simulation_time}")
    """

    # Create a dictionary to store paths traveled and time in memory for each frame
    frame_stats = {frame.frame_id: {'num_nodes_travelled': len(frame.path_traveled),
                                    'time_in_memory': frame.time_in_storage,
                                    'times_entered_memory': frame.num_times_entered_storage,
                                    'time_in_queue': frame.time_in_queue} for frame in frames}

    # Print the frame stats
    """
    for frame_id, stats in frame_stats.items():
        print(f"Frame {frame_id}:")
        print(f"  Num nodes travelled: {stats['num_nodes_travelled']}")
        print(f"  Time in memory: {stats['time_in_memory']}")
    """

    return frame_stats, simulation_time



# Main code to run the simulation

max_packets = 3  # Number of packets to generate and process
interval = 2  # Time interval between packet generations
num_nodes = 4  # Number of nodes in the linear graph
sender = 0  # Index of the sender node
receiver = 3  # Index of the receiver node
probability = 0.9  # Probability of a packet being 'in transit'

frame_stats, simulation_time = run_simulation(max_packets, interval, num_nodes, sender, receiver, print_status=True)


for frame_id, stats in frame_stats.items():
    print(f"Frame {frame_id}:")
    print(f"  Time in memory: {stats['time_in_memory']}")
    print(f"  Time in queue: {stats['time_in_queue']}")
    print(f" times_entered_memory' {stats['times_entered_memory']}")




Time 2: Frame 1 created at Node_0, destined for Node_3
Time 2: Frame 1 spent 0 time units in queue at Node_0

Time 2: Frame 1 forwarded from Node_0 to Node_1
Time 2: Frame 1 spent 0 time units in queue at Node_1
FRAME 1, GUARD TIME 3

Time 4: Frame 2 created at Node_0, destined for Node_3
Time 4: Frame 2 spent 0 time units in queue at Node_0

Time 4: Frame 2 forwarded from Node_0 to Node_1

Time 6: Frame 3 created at Node_0, destined for Node_3

Time 6: Frame 1 forwarded from Node_1 to Node_2
Time 6: Frame 2 spent 2 time units in queue at Node_1
FRAME 2, GUARD TIME 1
Time 6: Frame 3 spent 0 time units in queue at Node_0
Time 6: Frame 1 spent 0 time units in queue at Node_2

Time 6: Frame 3 forwarded from Node_0 to Node_1
FRAME 1, GUARD TIME 3

Time 8: Frame 2 forwarded from Node_1 to Node_2
Time 8: Frame 3 spent 2 time units in queue at Node_1
FRAME 3, GUARD TIME 1

Time 10: Frame 1 forwarded from Node_2 to Node_3

Time 10: Frame 3 forwarded from Node_1 to Node_2
Time 10: Frame 2 spen

## Rate of Bell Pair Distribution

In [3]:
num_frames = len(frame_stats)
c = 2e8 # speed of light in fiber
d_between_nodes = 100e3


def fiber_loss(a,d):
    """
    Parameters:
    a = attentuation coefficient of the optical fiber (dB/km)
    d = distance travelled in fiber (km)
    Returns:
    Loss in from fiber decibels (dB)
    """
    return  d * a

def memory_loss(a, t, avg_times_frame_enters_storage):
    '''
    Parameters:
    a = attenutation coefficient of the fiber storage line (dB/km)
    t = time in storage (s)
    Returns:
    Loss from memory in decibels (dB) with a minimum insertion loss of 4dB
    '''
    minimum_insertion_loss = 4
    avg_insertion_loss = minimum_insertion_loss * avg_times_frame_enters_storage
    return t * c * a + avg_insertion_loss


# avergae number of times a packet enters storage
total_times_packets_enters_storage = sum(stats['times_entered_memory'] for stats in frame_stats.values())
avg_times_frame_enters_storage = total_times_packets_enters_storage / num_frames if frame_stats else 0

# calculating average distance travelled by the frames
total_nodes = sum(stats['num_nodes_travelled'] for stats in frame_stats.values())
avg_nodes = total_nodes / num_frames if frame_stats else 0
avg_d = d_between_nodes * (avg_nodes - 1)

# calculating average time spent in storage
total_time_in_memory = sum(stats['time_in_memory'] for stats in frame_stats.values())
avg_mem_t = total_time_in_memory / num_frames

# Assuming the initial quantum payload contains 100 bell pairs
bp_per_frame_initial = 1e3
a_fiber = 1e-6
a_memory = 0.1

loss = fiber_loss(a_fiber, avg_d) + memory_loss(a_memory, avg_mem_t, avg_times_frame_enters_storage)
avg_bp_per_frame = bp_per_frame_initial * 10 ** (-loss / 10)

rate_bp_received = num_frames * avg_bp_per_frame / simulation_time


# now that we
rate_bp_received
avg_bp_per_frame

0.0